In [1]:
import sys

import numpy as np
import matplotlib.pyplot as plt

#import cleverhans
#from cleverhans.attacks_tf import fgsm
#from cleverhans.utils_tf import batch_eval

from _context import dl_uncertainty
from dl_uncertainty.models import ResNet, BlockStructure
from dl_uncertainty.data_utils import Cifar10Loader, Dataset
from dl_uncertainty.visualization import compose, Viewer
import dl_uncertainty.dirs

%matplotlib inline

In [2]:
ds_test = Cifar10Loader.load_test()
ds_test = ds_test.split(0, 4096)[0]
images_outof = list(map(np.flipud, ds_test.images))
ds_test_outof = Dataset(images_outof, ds_test.labels, ds_test.class_count)

In [3]:
zagoruyko_depth, k = 28, 10
group_count = 3
ksizes = [3, 3]
blocks_per_group = (zagoruyko_depth - 4) // (group_count * len(ksizes))
print(f"group count: {group_count}, blocks per group: {blocks_per_group}")
group_lengths = [blocks_per_group] * group_count
model = ResNet(
    input_shape=ds_test.input_shape,
    class_count=ds_test.class_count,
    batch_size=128,
    block_structure=BlockStructure.resnet(
        ksizes=ksizes, dropout_locations=[0]),
    group_lengths=group_lengths,
    base_width=k * 16,
    weight_decay=5e-4,
    training_log_period=39)
assert zagoruyko_depth == model.zagoruyko_depth, "invalid depth (zagoruyko_depth={}!={}=model.zagoruyko_depth)".format(
    zagoruyko_depth, model.zagoruyko_depth)

saved_path = dirs.SAVED_NETS + '/wrn-28-10-train-val/2018-04-22-0517/ResNet'  # vanilla
model.load_state(saved_path)

group count: 3, blocks per group: 4
(32, 32, 3) (32, 32, 160) resnet/conv/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block0/block/conv0/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block0/block/conv1/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block1/block/conv0/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block1/block/conv1/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block2/block/conv0/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block2/block/conv1/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block3/block/conv0/convolution:0
(32, 32, 160) (32, 32, 160) resnet/group0/block3/block/conv1/convolution:0
(32, 32, 160) (16, 16, 320) resnet/group1/block0/block/conv0/convolution:0
(16, 16, 320) (16, 16, 320) resnet/group1/block0/block/conv1/convolution:0
(16, 16, 320) (16, 16, 320) resnet/group1/block1/block/conv0/convolution:0
(16, 16, 320) (16, 16, 320) resnet/group1/block1/block/conv1/convolution:0
(16, 16, 320

TypeError: reduce_mean() got an unexpected keyword argument 'keepdims'

In [ ]:
values_in ={'max':[],'nonmax':[], 'sorted':[], 'sums':[]}
values_outof ={'max':[],'nonmax':[], 'sorted':[], 'sums':[]}

def add_logit_values(ds, values):
    cost, ev = model.test(ds)
    for im, lab in ds:
        logits = model._run(model.nodes.logits, [im], [lab])[0]
        sorted_logits = np.sort(logits)[::-1]
        values['max'].append(sorted_logits[0])
        values['nonmax'].extend(sorted_logits[1:])
        values['sorted'].append(sorted_logits)

add_logit_values(ds_test, values_in)
add_logit_values(ds_test_outof, values_outof)

In [ ]:
r = (np.min(values_in['nonmax']+values_outof['nonmax']), np.max(values_in['max']+values_outof['max']))

plt.hist(values_in['max'], bins=40, range=r, alpha=0.5, label='CIFAR-10-test') 
plt.hist(values_outof['max'], bins=40, range=r, alpha=0.5, label='CIFAR-10-test-UD')
plt.title("[Max logit] distribution") 
plt.xlabel('value')
plt.ylabel('frequency')
plt.legend()
plt.show()

plt.hist(values_in['nonmax'], bins=40, range=r, alpha=0.5, label='CIFAR-10-test') 
plt.hist(values_outof['nonmax'], bins=40, range=r, alpha=0.5, label='CIFAR-10-test-UD')
plt.title("[Non-max logit] distribution") 
plt.xlabel('value')
plt.ylabel('frequency')
plt.legend()
plt.show()

plt.hist(np.concatenate(values_in['sorted']), bins=40, range=r, alpha=0.5, label='CIFAR-10-test') 
plt.hist(np.concatenate(values_outof['sorted']), bins=40, range=r, alpha=0.5, label='CIFAR-10-test-UD')
plt.title("[Any logit] distribution") 
plt.xlabel('value')
plt.ylabel('frequency')
plt.legend()
plt.show()

for values, label in [(values_in, 'CIFAR-10-test'), (values_outof, 'CIFAR-10-test-UD')]:
    plt.scatter(
        [sl[0] for sl in values['sorted']],
        [np.sum(sl) for sl in values['sorted']], 
        alpha=0.1, s=2, edgecolors='none', label=label)
plt.xlabel('max(logits)')
plt.ylabel('sum(logits)')
plt.legend()
plt.show()

for values, label in [(values_in, 'CIFAR-10-test'), (values_outof, 'CIFAR-10-test-UD')]:
    plt.scatter(
        [sl[0] for sl in values['sorted']],
        [sl[1] for sl in values['sorted']], 
        alpha=0.5, s=2, edgecolors='none', label=label) 
plt.xlabel('max(logits)')
plt.ylabel('sorted_logits[1]')
plt.legend()
plt.show()

for values, label in [(values_in, 'CIFAR-10-test'), (values_outof, 'CIFAR-10-test-UD')]:
    plt.scatter(
        [sl[0] for sl in values['sorted']],
        [sl[1]+sl[2]+sl[3] for sl in values['sorted']], 
        alpha=0.5, s=2, edgecolors='none', label=label) 
plt.xlabel('max(logits)')
plt.ylabel('sum(sorted_logits[1:3])')
plt.legend()
plt.show()